# Identifikace systému, odhad parametrů modelu

Tento notebook je interaktivní, každou buňku můžete vykonat stisknutím `Shift-Enter`, můžete změnit její obsah a opětovně ji vykonat.

V kódu jsou tzv. elipsy, buď jako tři tečky `...` nebo podtržítko `_` v místě, kdě je potřeba doplnit nebo naprogramovat část dle zadání úkolu.

## Příprava v Modelice

- na fyzické mašině, 
- update `Bodylight-notebooks`, pomocí 
```
cd Bodylight-notebooks
git stash
git pull
```
- naklonujte si `Physiolibrary-models` nejlépe do stejného adresáře jako je Bodylight-notebooks nebo Bodylight-VirtualMachine 
```
cd ..
git clone https://github.com/creative-connections/Physiolibrary-models`
```

- ve virtuální mašině v OMEdit (nebo Dymola) otevřete `/vagrant_data/Physiolibrary_models/Metabolism/package.mo`
- vytvořte GlucoseToleranceTest1, který v čase 50h od začátku simulace bude pumpovat glukózu rychlostí 100 g za hodinu po dobu 20 minut, přepočtěte na mg/s a spojte je s GIRegulationBlock, simulujte 3 dny, zobrazte koncetraci glukózy a inzulínu (out1 a out2)
- vytvořte GlucoseToleranceTest2, který v čase 50h od začátku simulace bude pumpovat glukózu rychlostí 100 g za hodinu po dobu 20 minut, přepočtěte na kg/s a spojte je s GIRegulationComponent, simulujte 3 dny
- zkuste měnit parametry beta a nu (na 10% původní hodnoty), simulujte 3 dny
- exportujte GlucoseToleranceTest2 od FMU (mód ME), soubor dejte do adresáře `Seminar8FmiIdentification/fmus/`
 


## 1 Problém

Budeme zpracovávat data měření tzv. glukózového tolerančního testu. Máme informace, že pacient dostával intravenózně glukózu do krve rychlostí 28 mg/s. po dobu 12 minut. Pak byly v 12 minutových intervalech prováděny testy koncentrace glukózy a inzulínu hodnoty jsou v souborech `data/PatientX_y.csv` kde `X` je označení pacienta a `y` je buď `g`lukóza nebo `i`nzulín.


In [ ]:
# ukol 1.1 doplnit relativni cestu a soubor s daty glukozy pro pacienta A - data/PatientA_g.csv
import numpy
data = numpy.genfromtxt(_,delimiter=',')
data

In [ ]:
# casovou osu vygenerujeme, dle zadani to jde po 12 minutach (tj. po 720 sekundach)
# ukol 1.2 vyplnte po 720 pole time
time = numpy.arange(0,_*data.size,_)
time

In [ ]:
# definujeme rutinu pro kresleni grafu
def plot(x,y,z=None,labely=None,labelz=None):
    %matplotlib inline
    import matplotlib.pyplot as plt
    fig = plt.figure()
    plt.plot(x,y,label=labely,color='red')
    if z is not None:
        plt.plot(x,z, label=labelz,color='blue')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# ukol 1.3 vykreslete na ose x time, na ose y data
plot(_,_,labely="data patient A glucose")

## 2. FmPy nebo PyFMI knihovny a simulace modelu v Pythonu


Model z Modeliky exportovat jako FMU (nejlépe verze 2.0 v módu ModelExchange nebo Co-Simulation). V OpenModelice 1.16.x doporučujeme exportovat v módu ModelExchange. V Dymole doporučujeme exportovat v módu Co-Simulation (solver DASSL je robustní).

Knihovny FmPy a PyFMI zjednodušují simulace FMU v Pythonu. FmPy implementuje solvery v Pythonu. Knihovna PyFMI používá implementace z jiných knihoven v C++, které jsou obecně rychlejší.

V dalším textu budeme používat FmPy.

In [ ]:
# ukol 2.1 doplnte integracni krok simulatoru na 12 minut (prepoctete na sekundy) a do model1 nastavte cestu k exportovanemu FMU v adresari fmu/Metabolism.GlucoseInsulin.Test.GlucoseToleranceTest2.fmu
# nebo pouzijte Metabolism.GlucoseInsulin.Test.GlucoseToleranceTest.fmu pripraveny pro jistotu
import fmpy
model1 = _
result1 = result = fmpy.simulate_fmu(
    model1,
    start_time=0,
    stop_time=259200,
    step_size=_,
    record_events=False,
    output=['glucoseInsulinRegulation.glucoseConc','glucoseInsulinRegulation.insulinConc'],
    start_values={'glucoseInsulinRegulation.tissueUtilizationInsulinDependent.Nu_permu': 139000,'glucoseInsulinRegulation.insulinProduction.beta_mu': 1430},
    solver='CVode')
fmpy.util.plot_result(result1)

Jestli se objeví chyba, je třeba nainstalovat fmpy, např.: `conda install -c conda-forge fmpy`.

## 3. Kalibrace dat a simulace pro odhad parametrů


In [ ]:
# ukol 3.1 vykreslete do grafu na x time a na y glucoseInsulinRegulation.glucoseConc z predchozi simulace result1
# tip result1['...'] vybere patricne pole s hodnotami
plot(_,_,labely='model')

In [ ]:
# ukol 3.2 opet vykreslete data ze souboru na ose x time a na y data
plot(_,_,labely="data patient A glucose")

3.1 data a simulace modelu jsou vzorkována stejně? 

Ano - interval 12 minut (720s)

3.2 Odpovídají vzorky simulace modelu odpovídají vzorkům dat podle simulovaného experimentu? 

In [ ]:
print('data size',time.size,' model simulation size:',result1['time'].size)

Musíme vybrat začátek, kde simulace modelu začne odpovídat tomu co se děje v reálných datech

In [ ]:
# ukol 3.3 vyberte index odkud data simulace odpovidaji realnym datum, tip zacnete od 240 a zkousejte zvysovat
# az se grafy budou menit ve stejnou dobu
mycindex = _
model = result1['glucoseInsulinRegulation.glucoseConc'][mycindex:mycindex+10]
plot(time,data,model,labely='real data',labelz='model')

In [ ]:
# ukol 3.4 nastavte kalibrovany index na hodnotu odhadnutou v predchozim kroku
cindex = _ # calibrating index

In [ ]:
# Porovnani modelu a simulace jako funkce
def simulatemodelbeta(relativebeta=1,modelfile=model1):
    # odsimulovat s parametrem beta*relativebeta
    modelresult = fmpy.simulate_fmu(
        modelfile,
        start_time=0,
        stop_time=259200,
        step_size=720,
        record_events=False,
        output=['glucoseInsulinRegulation.glucoseConc','glucoseInsulinRegulation.insulinConc'],
        start_values={
            'glucoseInsulinRegulation.tissueUtilizationInsulinDependent.Nu_permu': 139000,
            'glucoseInsulinRegulation.insulinProduction.beta_mu': 1430*relativebeta
        },
        solver='CVode')
    return modelresult;

def comparemodeldatabeta(relativebeta=1,realdata=data,realtime=time,modelfile=model1):
    modelresult = simulatemodelbeta(relativebeta,modelfile)
    # vykreslit do grafu
    plot(realtime,realdata,modelresult['glucoseInsulinRegulation.glucoseConc'][cindex:cindex+10],labely='real data',labelz='model')
    



In [ ]:
# ukol 3.5 porovnejte model a data pro ruzne parametry (0.1,0.2,0.5,0.9)
comparemodeldatabeta(_)

Zkusíme najit hrubou silou parametr `beta` tak, aby model odpovídal co nejvíc datům - rozdíl mezi modelem a daty byl minimální.

Nejprve definujeme metriku rozdílu mezi simulací modelu a daty, tj. účelová funkce (objective function) $ y=\sum_{i=1}^n (s_i - d_i)^2 $ kde $s_i$ je hodnota sledované veličiny simulace v bodu $i$ a $d_i$ je hodnota sledované veličiny naměřených dat.



In [ ]:
# ukol 3.6 doplnte do sumy vypocet, 
# tip1 s,d jsou pole, k prvku pole pristupuji pomoci hranatych zavorek, 
# tip2 mocnina cisla x se zapise jako napr. x**2
def mydiff(s,d):
    sum = 0
    for i in range(0,s.size):
        sum+=_
    return sum

Nyní definujeme cyklus, který odsimuluje model s parametry 1 promile až 1000 promile normální hodnoty beta.

In [ ]:
# ukol 3.7 zkuste odsimulovat pro promile - 1000 moznych hodnot - odkomentujte
# hrubou silou odsimulujeme vsech 1000 moznych hodnot
#diffs = [] # pole s hodnotou ucelove funkci a parametrem
#for i in range(1,1000):
#    modelresult = simulatemodelbeta(i/1000)
#    diff= mydiff(modelresult['glucoseInsulinRegulation.glucoseConc'][cindex:cindex+10],data)
#    diffs.append([i/1000,diff])
# v poli diffs mam nyni parametr a hodnotu ucelove funkce (vzdalenost mezi simulaci a realnymi daty)

In [ ]:
# ukol 3.8 pokud predchozi vypocet spadl (kernel dead), zakomentujte predchozi bunku a pustte tuto
# varianta, pokud predchozi vypocet spadl, hrubou silou zkusime procenta
diffs = [] # pole s hodnotou ucelove funkci a parametrem
for i in range(1,100):
    modelresult = simulatemodelbeta(i/100)
    diff= mydiff(modelresult['glucoseInsulinRegulation.glucoseConc'][cindex:cindex+10],data)
    diffs.append([i/100,diff])


In [ ]:
# muzeme zobrazit jak procento beta ovlivnuje rozdil mezi daty a modelem
ndiffs = numpy.array(diffs)
plot(ndiffs[:,0],ndiffs[:,1],labely="diff between data and model")

Hodnota parametru s nejnizsi hodnotou ucelove funkce je hledany odhad parametru, ktery odpovida realnym datum.

In [ ]:
# setridime pole diffs podle 2. prvku (hodnota ucelove funkce)
diffs.sort(key=lambda x:x[1])

Nejlepsi shoda modelu s daty je prvni (v Pythonu nulty prvek setrideneho pole).

In [ ]:
modelbeta = diffs[0][0]
print('Nejlepsi shoda modelu s daty je pro parametr beta (v podilu k normalni hodnote)= ',modelbeta)

In [ ]:
# ukol 3.9 jak vypada srovnani dat a modelu
comparemodeldatabeta(_)

In [ ]:
# ukol 3.10 jak vypada srovnani dat a modelu s 2 nejlepsi hodnotou parametru? Tip diffs[x][0]
print(_)
comparemodeldatabeta(_)

In [ ]:
# Ukol 3.11 Nic se nemusi doplnit, jen jaky je odhad parametr nu pro ty sama data, pokud necham parametr beta v norme?
# Porovnani modelu a simulace jako funkce
def simulatemodelnu(relativenu=1,modelfile=model1):
    # odsimulovat s parametrem beta*relativebeta
    modelresult = fmpy.simulate_fmu(
        modelfile,
        start_time=0,
        stop_time=259200,
        step_size=720,
        record_events=False,
        output=['glucoseInsulinRegulation.glucoseConc','glucoseInsulinRegulation.insulinConc'],
        start_values={
            'glucoseInsulinRegulation.tissueUtilizationInsulinDependent.Nu_permu': 139000*relativenu,
            'glucoseInsulinRegulation.insulinProduction.beta_mu': 1430
        },
        solver='CVode')
    return modelresult;
def comparemodeldatanu(relativenu=1,realdata=data,realtime=time,modelfile=model1):
    modelresult = simulatemodelnu(relativenu,modelfile)
    # vykreslit do grafu
    plot(realtime,realdata,modelresult['glucoseInsulinRegulation.glucoseConc'][cindex:cindex+10],labely='real data',labelz='model')
diffs2 = [] # pole s hodnotou ucelove funkci a parametrem
for i in range(1,100):
    modelresult = simulatemodelnu(i/100)
    diff2= mydiff(modelresult['glucoseInsulinRegulation.glucoseConc'][cindex:cindex+10],data)
    diffs2.append([i/100,diff2])
# muzeme zobrazit jak procento beta ovlivnuje rozdil mezi daty a modelem
ndiffs2 = numpy.array(diffs2)
plot(ndiffs2[:,0],ndiffs2[:,1],labely="diff between data and model(nu)")    
diffs2.sort(key=lambda x:x[1])
print('Nejlepsi shoda dat a modelu s parametrem nu v procentech s normou= ',diffs2[0][0])


# 4. Interpretace

Snížená hodnota (10% normy) parametru beta v modelu může svědčit pro diagnózu diabetu 1.typu (snížená produkce inzulínu beta buněk).

Snížená hodnota (10% normy) parametru nu v modelu může svědčit pro diagnózu diabetu 2.typu (snížená citlivost tkání na inzulin)

Musíme vzít v potaz i naměřená data hladiny inzulinu během glukózového tolerančního testu.


In [ ]:
datai = numpy.genfromtxt('data/PatientA_i.csv',delimiter=',')

In [ ]:
plot(time,datai,labely='insulin data')

In [ ]:
# zobrazime hladinu insulinu modelu, v normalnim fyziologickem stavu
plot(result1['time'],result1['glucoseInsulinRegulation.insulinConc'],labely='model')

In [ ]:
# ukol 4.1 nastavte nu na hodnotu odhadnutou v ukolu 3.11
result2 = simulatemodelnu(_)

In [ ]:
plot(result2['time'],result2['glucoseInsulinRegulation.insulinConc'],labely='model')

In [ ]:
plot(time,datai,result2['glucoseInsulinRegulation.insulinConc'][cindex:cindex+10],labely='real data',labelz='model')

In [ ]:
result2['glucoseInsulinRegulation.insulinConc'][cindex:cindex+10]

In [ ]:
# Má pacient A diabetes a jakého pravděpodobně typu podle naměřených dat

# 5. Další úkoly

Úkol 5.1 a 5.2 Určete parametry beta nebo nu pro pacienty B a C, data v podadresari `data` a odhadněte zda-li mají diabetes a jakého typu?

# 6. Jiné metody nez hrubá síla

- Monte Carlo - ze souboru (1000x1000) možných hodnot vyberu náhodně podmnožinu hondot, vyberu tu která se nejvíc blíží reálným datům  
- Úkol 6.1 zkuste nahradit hrubou sílu optimalizačním algoritmem `differential_evolution` z balíčku`scipy.optimize`